In [165]:
from collections import deque
def bfs(startstate):
    global nrExplored
    frontier =  deque([startstate])
    prev = {startstate:None}
    while frontier:
        state = frontier.popleft()  
        nrExplored+=1
        print(state, end=' ')                        
        if goaltest(state):
            #print(len(prev),len(frontier))
            return prev, state
        for v in nextstates(state):
            if v not in prev:
                frontier.append(v)
                prev[v] = state
    return None, None
                
def dfs(startstate):
    frontier =  [startstate]
    prev = {startstate:None}
    while frontier:
        state = frontier.pop()   
        print(state, end=' ')     
        if goaltest(state):
            #print(len(prev),len(frontier))
            return prev,state
        for v in reversed(nextstates(state)):                  # nach rechts runter, für links: reversed(nextstates(state))
            if v not in prev:
                frontier.append(v)
                prev[v] = state
    return None, None

def reconstructPath(prev,goalstate):
    state = goalstate
    path = []
    while state is not None:
        path.append(state)
        state = prev[state]        
    path.reverse()
    return path


In [170]:
def nextstates(state):
    ''' returns: Liste mit den möglichen Folgezuständen '''
    if state in baum:
        return baum[state]
    else: return []

def goaltest(state):
    return state == 18

 
baum = {1:[12,6,7], 12: [14,22,10], 6: [4,8], 7: [2,18], 22:[3,9], 2:[13]}
prev, goalstate = dfs(1)
print()
print(reconstructPath(prev,goalstate))
 

1 12 14 22 3 9 10 6 4 8 7 2 13 18 
[1, 7, 18]


In [167]:
def swap(state,i,j):
    '''
    state: tuple
    i,j: gueltige Indizes für das tuple
    returns: tuple mit vertauschten Werten an Positionen i und j
    '''
    tmp = list(state)
    tmp[i],tmp[j] = tmp[j],tmp[i]
    return tuple(tmp)

def show(state):
    for i in range(9):
        if i%3==0:
            print()
        print(state[i],end = ' ')
    print()

def nextstates(state):
    tmp = []
    i0 = state.index(0)
    for j in tauschDict[i0]:
        tmp.append(swap(state,i0,j))
    return tmp

def goaltest(state):
    return state == (0,1,2,3,4,5,6,7,8) 

In [ ]:
%%time
nrExplored = 0
tauschDict = {0:[1,3],1:[2,4,0],2:[5,1],3:[4,6,0],4:[5,7,3,1],5:[8,4,2],6:[7,3],7:[8,6,4],8:[7,5]}
startstate = (7,2,4,5,0,6,8,3,1)   
#startstate = (8,2,7,3,4,6,5,0,1)
#startstate = (1,2,3,4,5,6,8,7,0)    # nicht lösbar
prev, goalstate = bfs(startstate)
if prev is None:
    print('nicht lösbar')
else:
    path = reconstructPath(prev,goalstate)
    print(f'Lösung in {len(path)-1} Zügen gefunden.')

print(nrExplored) 

Das dictionary *prev* dient mehreren Zwecken:
 - wir merken uns, was wir schon untersucht haben (explored) oder was noch zur Untersuchung vorgesehen ist (frontier)
 - wir merken uns den Elternknoten

In [133]:
for state in path:
    show(state)


7 2 4 
5 0 6 
8 3 1 

7 2 4 
0 5 6 
8 3 1 

0 2 4 
7 5 6 
8 3 1 

2 0 4 
7 5 6 
8 3 1 

2 5 4 
7 0 6 
8 3 1 

2 5 4 
7 6 0 
8 3 1 

2 5 4 
7 6 1 
8 3 0 

2 5 4 
7 6 1 
8 0 3 

2 5 4 
7 6 1 
0 8 3 

2 5 4 
0 6 1 
7 8 3 

2 5 4 
6 0 1 
7 8 3 

2 5 4 
6 1 0 
7 8 3 

2 5 4 
6 1 3 
7 8 0 

2 5 4 
6 1 3 
7 0 8 

2 5 4 
6 1 3 
0 7 8 

2 5 4 
0 1 3 
6 7 8 

2 5 4 
1 0 3 
6 7 8 

2 5 4 
1 3 0 
6 7 8 

2 5 0 
1 3 4 
6 7 8 

2 0 5 
1 3 4 
6 7 8 

0 2 5 
1 3 4 
6 7 8 

1 2 5 
0 3 4 
6 7 8 

1 2 5 
3 0 4 
6 7 8 

1 2 5 
3 4 0 
6 7 8 

1 2 0 
3 4 5 
6 7 8 

1 0 2 
3 4 5 
6 7 8 

0 1 2 
3 4 5 
6 7 8 
